# ETL

# 1. Importar librerias

In [1]:
import pandas as pd
import numpy as np
from Funciones import contar_valores_sd
from Funciones import data_cleaned
import warnings
warnings.filterwarnings("ignore", message="Downcasting.*", category=FutureWarning)

# 2. Carga de Datos

In [2]:
# Leer el archivo homicidios.xlsx y cargar las hojas "Hechos" y "Víctimas" en DataFrames
df_homicidios_h = pd.read_excel("DataSets/homicidios.xlsx", sheet_name="HECHOS")
df_homicidios_v = pd.read_excel("DataSets/homicidios.xlsx", sheet_name="VICTIMAS")
df_lesiones_h = pd.read_excel("DataSets/lesiones.xlsx", sheet_name="HECHOS")
df_lesiones_v = pd.read_excel("DataSets/lesiones.xlsx", sheet_name="VICTIMAS")

# 3. Exploracion y Limpieza de Datos

## 3.1 Dataset Homicidios - Hechos

### 3.1.1. Exploración inicial de los datos

In [3]:
#Vizualizo las primeras filas del DataFrame
df_homicidios_h.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [4]:
# Obtener información sobre las columnas y los tipos de datos
df_homicidios_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

### 3.1.2. Limpieza y Normalización de datos

In [5]:
# Usando la funcion contar_valores_sd para ver la cantidad y porcentaje de valores "SD"
contar_sd = contar_valores_sd(df_homicidios_h)
print(contar_sd)

           Columna  Cantidad de SD  Porcentaje de SD
0             HORA               1          0.143678
1               HH               1          0.143678
2  LUGAR_DEL_HECHO               1          0.143678
3          VICTIMA               9          1.293103
4          ACUSADO              23          3.304598


In [6]:
# Reemplazar "sd" y "SD" con NaN en todo el DataFrame
df_homicidios_h.replace(["sd", "SD"], np.nan, inplace=True)

In [7]:
# Contar filas vacías por columna
filas_vacias_por_columna = df_homicidios_h.isnull().sum()
print(filas_vacias_por_columna)

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    9
ACUSADO                   23
dtype: int64


En el conteo de filas vacias o nulos vemos que las columnas 'Altura' y 'Cruce' tienen significativamente una mayor cantidad de filas vacias, Lo recomendable es no usarlas y/o eliminarlas debido que no me aportan valor

In [8]:
df_homicidios_h['HH'] = df_homicidios_h['HH'].interpolate(method='linear')

In [9]:
# Contar filas duplicadas
duplicados_totales = df_homicidios_h.duplicated().sum()
print(duplicados_totales)

0


Como se logra ver nuestro DataFrame no tiene filas duplicadas

In [10]:
# Columnas para eliminar
columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

Decidi eliminar las columnas 'AAAA', 'MM', y 'DD' debido que tenemos una columna 'fecha' donde tenemos todo los datos de las 3 columnas anteriores. Tambien mantiene una estructura más estandarizada, ahorra espacio y facilita el análisis.

Tambien eliminamos la columna 'Hora' debido que la informacion que posee ya la tenemos en otra columna 'HH' y nos facilita el tipo de informacion para trabajar de forma mas rapida.

Así mismo las columnas 'LUGAR_DEL_HECHO','Calle','Dirección Normalizada', tienen datos que encontramos en direccion de los hechos y tambien 'XY (CABA)' que contienen coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de 'latitud' y 'longitud'.

In [11]:
# Renombrar columnas
rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

Renombramos columnas para mejorar el entendimiento de la columna(una forma más descrptiva)

In [12]:
# Valores de las sigueintes colummnas que convertiremos en minuscula
columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

Uso la funcion lower para normalizar los valores ,debido que observo los valores en mayuscula y es preferible trabajar todo en minuscula

In [13]:
# Convertir a entero
columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

# Categorización de columnas
columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado']

Convierti las columnas para una mayor facilidad al manipular los datos y tambien la compatibilidad para realizar funciones

In [14]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

df_homicidios_h_cleaned = data_cleaned(df_homicidios_h,
                                    strip_spaces = True, # Eliminar espacios en blanco                                  
                                    drop_columns = columns_to_drop,  # Eliminar columnas
                                    rename_columns = rename_dict,  # Renombrar columnas
                                    lowercase_columns = columns_to_lower, # Convertir a minuscula
                                    convert_to_int_columns = columns_to_int, # Conversión a entero
                                    categorize_columns = columns_to_categorize # Categorización de columnas
                                   )

In [15]:
df_homicidios_h_cleaned.head()

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros


In [16]:
df_homicidios_h_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_siniestro      696 non-null    object        
 1   nro_victimas      696 non-null    Int64         
 2   fecha             696 non-null    datetime64[ns]
 3   franja_hora       696 non-null    Int64         
 4   tipo_calle        696 non-null    category      
 5   comuna            696 non-null    Int64         
 6   longitud          696 non-null    object        
 7   latitud           696 non-null    object        
 8   vehiculo_victima  687 non-null    category      
 9   vehiculo_acusado  673 non-null    category      
dtypes: Int64(3), category(3), datetime64[ns](1), object(3)
memory usage: 43.2+ KB


## 3.2 Dataset Homicidios - Victimas

### 3.2.1. Exploración inicial de los datos

In [17]:
#Vizualizo las primeras filas del DataFrame
df_homicidios_v.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [18]:
# Obtener información sobre las columnas y los tipos de datos
df_homicidios_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


### 3.2.2. Limpieza y Normalización de datos

In [19]:
# Usando la funcion contar_valores_sd para ver la cantidad y porcentaje de valores "SD"
contar_sd = contar_valores_sd(df_homicidios_v)
print(contar_sd)

               Columna  Cantidad de SD  Porcentaje de SD
0                  ROL              11          1.534170
1              VICTIMA               9          1.255230
2                 SEXO               6          0.836820
3                 EDAD              53          7.391911
4  FECHA_FALLECIMIENTO              68          9.483961


In [20]:
# Reemplazar "sd" y "SD" con NaN en todo el DataFrame
df_homicidios_v.replace(["sd", "SD"], np.nan, inplace=True)

In [21]:
# Contar filas vacías por columna
filas_vacias_por_columna_v = df_homicidios_v.isnull().sum()
print(filas_vacias_por_columna_v)

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64


In [22]:
# Contar filas duplicadas
duplicados_totales = df_homicidios_v.duplicated().sum()
print(duplicados_totales)

0


Como se logra ver nuestro DataFrame no tiene filas duplicadas

In [23]:
# Columnas para eliminar
columns_to_drop = ['FECHA','AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO', 'VICTIMA']

Las columnas que estamos eliminando 'FECHA','AAAA', 'MM', 'DD', 'VICTIMA' ya tenemos en el DataFrame 'Hechos' y 'FECHA_FALLECIMIENTO' me parece irrelevante para el analisis

In [24]:
# Renombrar columnas
rename_dict = {'ID_hecho': 'id_siniestro', 'SEXO': 'sexo',
               'EDAD': 'edad', 'ROL': 'rol'}

Renombramos columnas para mejorar el entendimiento de la columna(una forma más descrptiva)

In [25]:
# Columnas para pasar a minúcula
columns_to_lower = ['SEXO','ROL']

In [26]:
# Convertir a entero
columns_to_int = ['edad']

# Categorización de columnas
columns_to_categorize = ['sexo','edad','rol']


Convierti las columnas para una mayor facilidad al manipular los datos y tambien la compatibilidad para realizar funciones

In [27]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 
df_homicidios_v_cleaned = data_cleaned(df_homicidios_v,
                                    strip_spaces = True, # Eliminar espacios en blanco                                   
                                    drop_columns = columns_to_drop,  # Eliminar columnas
                                    rename_columns = rename_dict,  # Renombrar columnas
                                    lowercase_columns = columns_to_lower, # Convertir a minúsculas 
                                    convert_to_int_columns = columns_to_int, #Conversión entero
                                    categorize_columns = columns_to_categorize # Categorización de columnas
                                    )

In [28]:
# Crear la columna 'graved' con el valor 'muerte'
df_homicidios_v_cleaned['gravedad'] = 'muerte'

# Definir las columnas a categorizar
categorize_columns = ['gravedad']

# Llamar a la función data_cleaned para categorizar la columna 'graved'
df_homicidios_v_cleaned = data_cleaned(df_homicidios_v_cleaned, categorize_columns=categorize_columns)

In [29]:
df_homicidios_v_cleaned.head()

,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19,muerte
1,2016-0002,conductor,masculino,70,muerte
2,2016-0003,conductor,masculino,30,muerte
3,2016-0004,conductor,masculino,18,muerte
4,2016-0005,conductor,masculino,29,muerte


In [30]:
df_homicidios_v_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id_siniestro  717 non-null    object  
 1   rol           706 non-null    category
 2   sexo          711 non-null    category
 3   edad          664 non-null    Int64   
 4   gravedad      717 non-null    category
dtypes: Int64(1), category(3), object(1)
memory usage: 14.6+ KB


## 4. Combinar Datos

In [31]:
df_homicidios = pd.merge(df_homicidios_h_cleaned, df_homicidios_v_cleaned, on='id_siniestro', how='inner')

In [32]:
df_homicidios.head()

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto,conductor,masculino,19,muerte
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros,conductor,masculino,70,muerte
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto,conductor,masculino,30,muerte
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN,conductor,masculino,18,muerte
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros,conductor,masculino,29,muerte


## 5. Modificaciones y Transfomaciones Adicionales

In [33]:
# Eliminar filas con valores nulos o vacíos
df_homicidios = df_homicidios.dropna()

# 6. Almacenamiento y Union de Datos

In [34]:
df_homicidios.to_csv('DataSets/homicidios_cleaned1.csv', index=False)

